In [ ]:
import pandas as pd
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split

-Need to do a sort of sentiment analysis like with word vectorization
-Load in other data things


https://www.kaggle.com/code/mehmetlaudatekman/tutorial-word-embeddings-with-svm

In [ ]:
shower_dp = 'data/showerThoughts.csv'
shower_data = pd.read_csv(shower_dp,header=0)
shower_data.head()

In [ ]:
shower_data.columns

In [ ]:
shower_data = shower_data.drop(['domain', 'id','permalink', 'over_18','link_flair_css_class',
       'author_flair_css_class', 'selftext','thumbnail', 'name', 'url', 'link_flair_text','distinguished'], axis='columns')
shower_data.columns

In [ ]:
plt.plot(shower_data['ups'], shower_data['downs'],color="r")
plt.xlabel("Up Votes")
plt.ylabel("Down Votes")
plt.title("Engagement")
plt.show()

In [ ]:
shower_data["engagement"] = shower_data["ups"] + shower_data["downs"] + shower_data["num_comments"]

In [ ]:
#August 20th 2013 date created


In [ ]:
shower_data

In [ ]:
import nltk
from nltk.tokenize import word_tokenize

In [ ]:
nltk.download(['punkt', "names", "stopwords", "averaged_perceptron_tagger", "vader_lexicon"])

In [ ]:
trial = shower_data['title']
tokens = trial.apply(lambda x: word_tokenize(x))
#tokenizing all of the 'title' column, need to now vectorize them// word embeddings

In [ ]:
stopwords = nltk.corpus.stopwords.words("english")
stopwords.append(".")
stopwords.append(",")

#Cleaning out stopwords
clean = []
for sent in tokens:
    new_sent = []
    for word in sent:
        if word.lower() not in stopwords:
            new_sent.append(word)
    clean.append(new_sent)
         

In [ ]:
def get_all_words(cleaned_tokens_list):
    for tokens in cleaned_tokens_list:
        for token in tokens:
            yield token

all_words = get_all_words(clean)

In [ ]:
from nltk import FreqDist

freq_dist_pos = FreqDist(all_words)
print(freq_dist_pos.most_common(10))


In [ ]:
import nltk.sentiment 
from nltk.sentiment import SentimentIntensityAnalyzer

In [ ]:
sia = SentimentIntensityAnalyzer()

In [ ]:
def sentiment_to_df(titles):
    neg = []
    neu = []
    pos = []
    for t in titles:
        scores = sia.polarity_scores(t)
        neg.append(scores['neg'])
        neu.append(scores['neu'])
        pos.append(scores['pos'])
        
    return neg, neu, pos

In [ ]:
neg, neu, pos = sentiment_to_df(shower_data['title'])
shower_data['neg'] = neg
shower_data['neu'] = neu
shower_data['pos'] = pos
shower_data
#TF-IDF

In [ ]:
features = shower_data[['neg', 'neu', 'pos']]
targets = shower_data['ups']

features_norm = (features-features.min())/(features.max()-features.min())
targets_norm = (targets-targets.min())/(targets.max()-targets.min())

In [ ]:
features

In [ ]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
scaler.fit(features)
feat = scaler.transform(features)
feat

In [ ]:
target_scaler = MinMaxScaler()
t = np.array(targets).reshape(-1, 1)
target_scaler.fit(t)

targ = target_scaler.transform(t)
targ

In [ ]:
from sklearn.preprocessing import PolynomialFeatures
poly = PolynomialFeatures(2)
features_poly = poly.fit_transform(features_norm)
f_poly = poly.fit_transform(feat)

In [ ]:
features_poly

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(f_poly, targ, test_size=0.2)

In [ ]:
y_train

In [ ]:
from sklearn import linear_model
from sklearn.metrics import mean_squared_error
reg = linear_model.BayesianRidge()
reg.fit(X_train, y_train)
preds = reg.predict(X_test)
mean_squared_error(y_test, preds)

In [ ]:
targets.iloc[0]

In [ ]:

#features.iloc[0].reshape(1, -1)
x = np.array(features.iloc[996].tolist()).reshape(1, -1)
x_poly = poly.fit_transform(x)
x_poly

In [ ]:
fun = reg.predict(x_poly)

In [ ]:
fun

In [ ]:
target_scaler.inverse_transform([fun])

In [ ]:
from tensorflow import keras
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.preprocessing.text import Tokenizer

In [ ]:
 # config values// Beginning of word embeddings
embed_size = 300 # how big is each word vector
max_features = 50000 # how many unique words to use (i.e num rows in embedding vector)
maxlen = 100 # max number of words in a question to use

In [ ]:
texts = shower_data['title']
targets_ups = shower_data['ups']
clean

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(clean, targ, test_size=.2, train_size=.8, random_state=None, shuffle=True, stratify=None)

tokenizer = tf.keras.preprocessing.text.Tokenizer(num_words=max_features)
tokenizer.fit_on_texts(X_train)
X_train = tokenizer.texts_to_sequences(X_train)
X_test = tokenizer.texts_to_sequences(X_test)

X_train = pad_sequences(X_train, maxlen=maxlen)
X_test = pad_sequences(X_test, maxlen=maxlen)

In [ ]:
X_train

In [ ]:
reg = linear_model.BayesianRidge()
reg.fit(X_train, y_train)
preds = reg.predict(X_test)
mean_squared_error(y_test, preds)

In [ ]:
shower_data['ups'].describe